In [1]:
%%time
import os
import json
import warnings
from math import sqrt
import ipfshttpclient
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from web3 import Web3
from keras import metrics
from keras.layers import GRU, Input, RepeatVector, concatenate, Activation, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from pandas import read_csv
from numpy import split, array
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error, mean_squared_log_error
from tensorflow.keras.models import Model, Sequential, model_from_json
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, RepeatVector, TimeDistributed
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.metrics import RootMeanSquaredError, MeanSquaredError, MeanAbsoluteError

warnings.filterwarnings("ignore")

CPU times: total: 2.2 s
Wall time: 6.97 s


In [2]:
%%time
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8573"))
print(w3.is_connected())
w3.eth.default_account=w3.eth.accounts[0]

True
CPU times: total: 15.6 ms
Wall time: 15.8 ms


In [3]:
%%time
abi1=[
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "modelCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_modelData2",
				"type": "string"
			}
		],
		"name": "setModelData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "dataCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_data2",
				"type": "string"
			}
		],
		"name": "setTrainingData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "accessListAddress",
				"type": "address"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "modelCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_modelData2",
				"type": "string"
			}
		],
		"name": "updateModelData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "dataCount",
				"type": "uint256"
			},
			{
				"internalType": "string",
				"name": "_data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "_data2",
				"type": "string"
			}
		],
		"name": "updateTrainingData",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "dataMap",
		"outputs": [
			{
				"internalType": "string",
				"name": "data1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "data2",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getModelData",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "modelData1",
						"type": "string"
					},
					{
						"internalType": "string",
						"name": "modelData2",
						"type": "string"
					}
				],
				"internalType": "struct Publish.ModelData",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "getNodeAddress",
				"type": "address"
			}
		],
		"name": "getNodeInfo",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			},
			{
				"internalType": "uint8",
				"name": "",
				"type": "uint8"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getTrainingData",
		"outputs": [
			{
				"components": [
					{
						"internalType": "string",
						"name": "data1",
						"type": "string"
					},
					{
						"internalType": "string",
						"name": "data2",
						"type": "string"
					}
				],
				"internalType": "struct Publish.TrainingData",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "modelMap",
		"outputs": [
			{
				"internalType": "string",
				"name": "modelData1",
				"type": "string"
			},
			{
				"internalType": "string",
				"name": "modelData2",
				"type": "string"
			}
		],
		"stateMutability": "view",
		"type": "function"
	}
]


CPU times: total: 0 ns
Wall time: 0 ns


In [4]:
%%time
address1 = "0x6041cc1c53F338281FB623b409Ebd90e40557f71"
Pub_contract = w3.eth.contract(address=address1, abi=abi1)

CPU times: total: 0 ns
Wall time: 5.74 ms


In [5]:
%%time
abi=[
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "accessListAddress",
				"type": "address"
			}
		],
		"stateMutability": "nonpayable",
		"type": "constructor"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "change_train_status",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [],
		"name": "configsCount",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "deleteConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			}
		],
		"name": "getConfiguration",
		"outputs": [
			{
				"components": [
					{
						"internalType": "uint256",
						"name": "n_past",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "n_features",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "n_future",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "lstm_units_1",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "lstm_units_2",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "dense_units_1",
						"type": "uint256"
					},
					{
						"internalType": "uint256",
						"name": "dense_units_2",
						"type": "uint256"
					},
					{
						"internalType": "bool",
						"name": "train",
						"type": "bool"
					}
				],
				"internalType": "struct Config_Data.Configuration",
				"name": "",
				"type": "tuple"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "address",
				"name": "getNodeAddress",
				"type": "address"
			}
		],
		"name": "getNodeInfo",
		"outputs": [
			{
				"internalType": "bool",
				"name": "",
				"type": "bool"
			},
			{
				"internalType": "uint8",
				"name": "",
				"type": "uint8"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "",
				"type": "uint256"
			}
		],
		"name": "modelConfigs",
		"outputs": [
			{
				"internalType": "uint256",
				"name": "n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "dense_units_2",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "train",
				"type": "bool"
			}
		],
		"stateMutability": "view",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "_configsCount",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "_train",
				"type": "bool"
			},
			{
				"internalType": "uint256",
				"name": "_n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_2",
				"type": "uint256"
			}
		],
		"name": "setConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	},
	{
		"inputs": [
			{
				"internalType": "uint256",
				"name": "index",
				"type": "uint256"
			},
			{
				"internalType": "bool",
				"name": "_train",
				"type": "bool"
			},
			{
				"internalType": "uint256",
				"name": "_n_past",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_features",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_n_future",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_lstm_units_2",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_1",
				"type": "uint256"
			},
			{
				"internalType": "uint256",
				"name": "_dense_units_2",
				"type": "uint256"
			}
		],
		"name": "updateConfiguration",
		"outputs": [],
		"stateMutability": "nonpayable",
		"type": "function"
	}
]
address = "0x3F2948E1faCc1F5B394eeb0D61B83A5fF07aaEcc"
Conf_contract = w3.eth.contract(address=address, abi=abi)

CPU times: total: 0 ns
Wall time: 12.4 ms


In [6]:
%%time
# Connect to IPFS
client = ipfshttpclient.connect("/ip4/127.0.0.1/tcp/5001/http")

CPU times: total: 0 ns
Wall time: 27.6 ms


In [7]:
%%time
H_u=Pub_contract.functions.getModelData(1).call()

CPU times: total: 15.6 ms
Wall time: 9.42 ms


In [8]:
%%time
H_u[1]

CPU times: total: 0 ns
Wall time: 0 ns


'QmdHq1yrQ7E8oXjbDD8j935jTpMxU6qNtDCmDgY4XMk7E7'

In [ ]:
#C=Conf_contract.functions.getConfiguration(1).call()

In [9]:
%%time
n_past=25
n_features=29
n_future=10

CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
%%time
df1 = pd.read_csv('unit16.csv', header=0,index_col=0)
df=df1.iloc[0:100000,:]

CPU times: total: 2.91 s
Wall time: 3.66 s


In [11]:
%%time
df=df.fillna(0)
df = df.iloc[:, :-4]

CPU times: total: 0 ns
Wall time: 14.7 ms


In [12]:
%%time
scaler = MinMaxScaler()
scaled = scaler.fit_transform(df.values)

CPU times: total: 0 ns
Wall time: 23.9 ms


In [ ]:
scaled

In [13]:
%%time
# Prepare the input and target data
def create_dataset(data, n_past, n_future):
    X, y = [], []
    for i in range(n_past, len(data) - n_future + 1):
        X.append(data[i - n_past:i])
        y.append(data[i:i + n_future, -1])
    return np.array(X), np.array(y)



X, y = create_dataset(scaled, n_past, n_future)


CPU times: total: 141 ms
Wall time: 223 ms


In [ ]:
X.shape,y.shape

In [14]:
%%time
# Get hashes
model_json_hash_u = H_u[0]
model_h5_hash_u = H_u[1]


CPU times: total: 0 ns
Wall time: 0 ns


In [15]:
%%time
import os

# Define the directory where you want to save the model and weights
save_dir = "C:\\Users\\montd\Desktop\\Project\\Deep Learning\\Updatable2"

CPU times: total: 0 ns
Wall time: 0 ns


In [16]:
%%time
# Retrieve the files from IPFS
model_json_file_u = client.cat(model_json_hash_u)
model_h5_file_u = client.cat(model_h5_hash_u)

CPU times: total: 15.6 ms
Wall time: 8.55 ms


In [17]:
%%time
# Save the retrieved model architecture
with open(os.path.join(save_dir, 'retrieved_model_u.json'), 'w') as file:
    file.write(model_json_file_u.decode('utf-8'))

# Save the retrieved model weights
with open(os.path.join(save_dir, 'retrieved_model_u.h5'), 'wb') as file:
    file.write(model_h5_file_u)

# Load the model architecture
with open(os.path.join(save_dir, 'retrieved_model_u.json'), 'r') as file:
    loaded_model_json = file.read()

# Create a model from the loaded architecture
loaded_model_u = model_from_json(loaded_model_json)

# Load the weights into the model
loaded_model_u.load_weights(os.path.join(save_dir, 'retrieved_model_u.h5'))

CPU times: total: 375 ms
Wall time: 460 ms


In [ ]:

loaded_model_u.summary()

In [18]:
%%time
predictions_u = loaded_model_u.predict(X)

3124/3124 [==============================] - 13s 4ms/step
CPU times: total: 15 s
Wall time: 14.2 s


In [19]:
%%time
predictions_u = np.squeeze(predictions_u)
np.shape(predictions_u)

CPU times: total: 0 ns
Wall time: 0 ns


(99966, 10)

In [20]:
%%time
H_l=Pub_contract.functions.getModelData(2).call()




CPU times: total: 0 ns
Wall time: 8.66 ms


In [21]:
%%time
# Get hashes
model_json_hash_l = H_l[0]
model_h5_hash_l = H_l[1]

CPU times: total: 0 ns
Wall time: 0 ns


In [22]:
%%time
# Retrieve files from IPFS
model_json_file_l = client.cat(model_json_hash_l)
model_h5_file_l = client.cat(model_h5_hash_l)

CPU times: total: 15.6 ms
Wall time: 8.62 ms


In [23]:
%%time
# Save files locally
with open('retrieved_model.json', 'w') as file:
    file.write(model_json_file_l.decode('utf-8'))

with open('retrieved_model.h5', 'wb') as file:
    file.write(model_h5_file_l)

CPU times: total: 0 ns
Wall time: 1.64 ms


In [24]:
%%time
import keras 
# Load model architecture from JSON
with open('retrieved_model.json', 'r') as json_file:
    loaded_model_json_l = json_file.read()

loaded_model_l = keras.models.model_from_json(loaded_model_json_l)

# Load weights into the model
loaded_model_l.load_weights("retrieved_model.h5")

CPU times: total: 344 ms
Wall time: 446 ms


In [ ]:
loaded_model_l.summary()

In [25]:
%%time
predictions_l = loaded_model_l.predict(X)

3124/3124 [==============================] - 13s 4ms/step
CPU times: total: 9.03 s
Wall time: 14.3 s


In [26]:
%%time
predictions_l = np.squeeze(predictions_l)
np.shape(predictions_l)

CPU times: total: 0 ns
Wall time: 0 ns


(99966, 10)

In [ ]:

np.shape(y)


In [27]:
mae = mean_absolute_error(y, predictions_u)
mse = mean_squared_error(y, predictions_u)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 0.0075216870942397375
Mean Squared Error: 7.972567766897402e-05
Root Mean Squared Error: 0.008928923656800635


In [28]:
mae = mean_absolute_error(y, predictions_l)
mse = mean_squared_error(y, predictions_l)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 0.0075216870942397375
Mean Squared Error: 7.972567766897402e-05
Root Mean Squared Error: 0.008928923656800635


In [29]:
%%time
c = 0.9
d = 0.1
weighted_avg = c*predictions_l + d*predictions_u

CPU times: total: 0 ns
Wall time: 3.21 ms


In [30]:
np.shape(weighted_avg)

(99966, 10)

In [31]:
mae = mean_absolute_error(y, weighted_avg)
mse = mean_squared_error(y, weighted_avg)
rmse = np.sqrt(mse)

print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 0.00752168027445402
Mean Squared Error: 7.972556766730921e-05
Root Mean Squared Error: 0.008928917496948284


In [33]:
np.shape(X),np.shape(good_predictions)

((99896, 25, 29), (99896, 10))

In [ ]:
type(X)

In [35]:
%%time
# Convert numpy arrays to lists
x_list = X.tolist()
#y_list = good_predictions.tolist()

CPU times: total: 9.73 s
Wall time: 11.1 s


In [36]:
type(x_list)

list

In [38]:
%%time
# Combine into one dictionary
data = {'x': x_list, 'y': [pred.tolist() for pred in good_predictions]}
# Write to JSON file
with open('data.json', 'w') as f:
    json.dump(data, f)

CPU times: total: 1min 26s
Wall time: 2min 2s


In [39]:
%%time
# Add file to IPFS
res1 = client.add('data.json')
print(res1)

<ipfshttpclient.client.base.ResponseBase: {'Name': 'data.json', 'Hash': 'QmUJuknScHeA5tV88ScgrYuPGDBSkgGJt1CA4uMDEuiUoZ', 'Size': '1487636166'}>
CPU times: total: 3.2 s
Wall time: 10.5 s


In [40]:
%%time
Pub_contract.functions.setTrainingData(2,res1['Hash'],res1['Hash']).transact()

CPU times: total: 15.6 ms
Wall time: 29.4 ms


HexBytes('0x9013c93128c214945ebfcddd6823bdd7c75dc31ec47744d7ab90f9f7716d5b2f')